#  Mida mínia informativa de finestra

En aquest _notebook_ de Jupyter exploraré quina és la mida mínima informativa per les finestres que analitzem amb les dades de Bell et al. 2020. Sabem que es poden fer mides de finestra més petites que a l'article original, pero a mesura que fem finestres més petites és evident que les mesures es tornen menys realistes.

Per fer-ho, compararé 3 mesures de  la taxa de recombinació en una finestra de mida concreta: la original del paper, que agafa el punt mig de la regió; la nostra, que agafa una mesura probabilstica; i la "real", per la qual se li assigna a cada fragment un punt concret que seria on hipotèticament va tenir lloc l'entrecreuament.



In [1]:
import pandas as pd
import random
import os
import numpy as np


## Especificació de variables


Per anar més ràpid, fare servir només el cromosoma 1 de l'individu 1. Més endavant si volem, es pot ampliar a més cromosomes i individus, ja que el filtratge es farà a partir de vectors. 

In [2]:
# Folders
outdir = "tmp/"
if not os.path.exists(outdir):
    os.mkdir(outdir)

# Subset inds and chr
individuals = ["NC1"]
chromosomes = ["chr1"]

# Windows
winsizes = [5000]
winsizes.extend(range(10000, 300000, 10000))
winsizes.extend(range(300000, 550000, 50000))
winsizes = [str(win) for win in winsizes]

# Amount of iterations
seeds = list(range(1,50, 1))


## Arxius amb crossovers

### Creació dels 3 modes de crossover

__Atenció! Aquesta part del codi no cal si ja existeixen els arxius!__

Els arxius existents són importat més endavant. 

Necessitem 3 tipus de crossover: en el centre de la regió, tota la regió, i un punt aleatori de la regió. Un cop generats aquests arxius, es poden reutilitzar pel que calgui, així que els guardarem a `data/use/avery_crossovers`. 

Aquí està la taula original, que servirà perl mètode probabilístic:

In [11]:
originalBed = pd.read_csv("../../data/use/avery_crossovers/allcrossovers.bed", sep = "\t", header = 'infer')

originalBed.head()

,chr,pos.leftbound,pos.rightbound,donor
0,chr1,779059,818802,NC9
1,chr1,779059,818954,NC9
2,chr1,779059,875756,NC9
3,chr1,779059,909894,NC9
4,chr1,779885,1158683,NC14


Taula agafant el centre de la regió:

In [12]:
# CENTER OF EVENTS
# This will not be the exact center in odd numbers, but works well enough
# -------------------------
centerBed = originalBed.copy()
centerBed['pos.leftbound'] = ( centerBed['pos.leftbound'] + centerBed['pos.rightbound'] )/2
centerBed['pos.leftbound'] = centerBed['pos.leftbound'].astype(int)
centerBed['pos.rightbound'] = centerBed['pos.leftbound']+1

centerBed.to_csv("../../data/use/avery_crossovers/allcrossovers_center.bed",index=None, sep='\t', mode='w')
centerBed.head()

,chr,pos.leftbound,pos.rightbound,donor
0,chr1,798930,798931,NC9
1,chr1,799006,799007,NC9
2,chr1,827407,827408,NC9
3,chr1,844476,844477,NC9
4,chr1,969284,969285,NC14


Taula agafant un lloc aleatori de cada regió:

In [3]:
# "REAL" EVENTS
# Let's suppose real events occured in the following positions:
# -------------------------
# This function creates a new random file

def newRandomCrossovers(randBed, outfile, seed):
    random.seed(seed)
    randlist = []

    for index, row in randBed.iterrows():
        randnum = random.randint(row[1], row[2]-1) # random.randint includes both numbers
        randlist.append(randnum)
    newtab  = randBed.copy()    
    newtab['pos.leftbound'] = randlist
    newtab['pos.rightbound'] = newtab['pos.leftbound']+1
    newtab.to_csv(outfile,index=None, sep='\t', mode='w') 
 

In [15]:
newRandomCrossovers(originalBed, "../../data/use/avery_crossovers/allcrossovers_random.bed", 1 )

In [ ]:
randBed = pd.read_csv("../../data/use/avery_crossovers/allcrossovers_random.bed", sep = "\t", header = 'infer')

### Filtratge dels 3 modes de crossover

Els 3 tipus de crossover els volem filtrar per només incloure aquells events dels individus i cromosomes que ens interessen, per anar més ràpid. 

In [4]:
def filter_individuals( bedfile, ind = individuals, chrom = chromosomes):
    # Save columns
    colnames = list(bedfile.columns) 
    # Filter individuals
    #bedfile['individual']=bedfile['donor_cell'].str.extract(r'(^NC\d*)_')
    bedfile = bedfile[bedfile['donor'].isin(ind)]
    # Filter chromosomes
    bedfile = bedfile[bedfile['chr'].isin(chrom)]
    # Remove extra columns
    bedfile = bedfile[colnames]
    # Finish
    return bedfile

In [5]:
# Original
originalBed = pd.read_csv("../../data/use/avery_crossovers/allcrossovers.bed", sep = "\t", header = 'infer')
originalBed = filter_individuals(bedfile = originalBed)
originalBed.to_csv(outdir+"allcrossovers_prob.bed",index=None, sep='\t', mode='w')
# Center
centerBed = pd.read_csv("../../data/use/avery_crossovers/allcrossovers_center.bed", sep = "\t", header = 'infer')
centerBed = filter_individuals(bedfile = centerBed)
centerBed.to_csv(outdir+"allcrossovers_center.bed",index=None, sep='\t', mode='w')
# Random
# randBed = filter_individuals(bedfile = randBed)
# randBed.to_csv(outdir+"allcrossovers_base.bed",index=None, sep='\t', mode='w')
# For more than one random file

for s in seeds:
    newRandomCrossovers(randBed = originalBed, outfile =  outdir+"allcrossovers_base"+str(s)+".bed", seed = s)
    

## Arxiu amb les finestres 

Existeix un arxiu que ens diu com de grans són els cromosomes, d'allà podem obtenir les mides dels cromosomes que voldrem fer servir. 



In [6]:
boundaries = pd.read_csv("../../data/use/assembly_hg38/minmax.strict.gff", sep = " ", header = None)
boundaries = boundaries[boundaries[0].isin(chromosomes)]
boundaries.to_csv(outdir+"region.bed",index=None, sep='\t', mode='w', header = None)

A continuació, apliquem el codi de fer les finestres: 

In [7]:
for w in winsizes:
    os.system('python ../../code/python/makeBedWindows.py --input '+outdir+'region.bed --output '+outdir+'windows_'+w+'.bed --fixedWindow '+w+' --windowMethod "fromLeft" --chromBoundaries '+outdir+'region.bed')

## Fer taxa de recombinació

Ara amb bedtools volem saber quins dels crossovers solapen amb les finestres. Després, makeRecRates calcula les taxes de recombinació i també fa el recompte, o sigui que és una taula que així com surti ja es podra comparar. 



In [8]:
modes = ["center", "prob"]
for s in seeds:
    modes.append("base"+str(s))
    
for w in winsizes:
    for m in modes: 
        os.system('bedtools intersect -wao -a "'+outdir+'windows_'+w+'.bed"  -b "'+outdir+'allcrossovers_'+m+'.bed" > "'+outdir+'comparison_'+w+'_'+m+'.txt"')
        os.system('python ../../code/python/makeRecRates.py --input "'+outdir+'comparison_'+w+'_'+m+'.txt" --output "'+outdir+'crossoverResult_'+w+'_'+m+'.txt" --numofsamples "../../data/use/avery_crossovers/numOfCells.txt"')

## Representar resultats

### Pujar les taules
    
Abans carregava les taules a la memòria pero ara per no petar-la, faré una funció que només caregui el que 
necessito.

In [9]:
tables = globals()
for w in winsizes:
    for m in modes:
        tables['rR'+w+m] = pd.read_csv( outdir+'crossoverResult_'+w+m+'.txt' ,  sep = "\t", header = 'infer')


In [9]:
def correl_calculation(winsizes, modes, seeds):
    correlations = []
    for s in seeds:
        for w in winsizes:
            base = pd.read_csv( outdir+'crossoverResult_'+w+'_base'+str(s)+'.txt' ,  sep = "\t", header = 'infer')
            for m in ["center", "prob"]:
                compare = pd.read_csv( outdir+'crossoverResult_'+w+'_'+m+'.txt' ,  sep = "\t", header = 'infer') 
                # Make correlation
                c1 = base['overlapScore'].corr(compare['overlapScore'])
                correlations.append({'winsize' : w, 'mode' : m, 'correl_count' : c1})
                # Return
    correlations = pd.DataFrame(correlations)
    return(correlations)

### Fer correlacions




In [10]:
correlations = correl_calculation(winsizes, ["center", "prob"], seeds )

In [11]:
# correlations

### Plot

In [12]:
correlations.to_csv("correlation.table",index=None, sep='\t', mode='a', header =  False )